# Northwind with the ipython-sql extension
You can use the ipython-sql extension to query the database live

In [1]:
import os
import sys
sys.path.append(os.path.realpath('..'))
from sqlalchemy import create_engine, MetaData
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image
from pandasql import sqldf
from sqlalchemy_schemadisplay import create_schema_graph
from configuration.config import DB_HOST #you will need to create this. It is not recommended to store any server credentials in a Github repo

In [3]:
connection_url = f"mssql+pymssql://{DB_HOST}/Northwind"

In [4]:
%load_ext sql

In [5]:
%sql $connection_url

## Orders by Country

In [20]:
%%sql 
SELECT 
    ShipCountry, 
    COUNT(OrderID) as NumOrders 
FROM 
    Orders 
GROUP BY 
    ShipCountry 
ORDER BY 
    NumOrders DESC;


 * mssql+pymssql://DESKTOP-80SJ5RC/Northwind
Done.


ShipCountry,NumOrders
USA,122
Germany,122
Brazil,83
France,77
UK,56
Venezuela,46
Austria,40
Sweden,37
Canada,30
Mexico,28


## Orders by City

In [21]:
%%sql
SELECT
	ShipCity,
	COUNT(OrderID) as NumOrders
FROM
	Orders
GROUP BY 
	ShipCity
ORDER BY
	NumOrders DESC

 * mssql+pymssql://DESKTOP-80SJ5RC/Northwind
Done.


ShipCity,NumOrders
Rio de Janeiro,34
London,33
Sao Paulo,31
Boise,31
Graz,30
Cunewalde,28
México D.F.,28
Cork,19
Bräcke,19
Albuquerque,18


## Orders By Year

In [22]:
%%sql
SELECT
	YEAR(OrderDate) as yearoforder,
	COUNT(OrderID) as NumOrders
FROM
	Orders
GROUP BY 
	YEAR(OrderDate)
ORDER BY
	yearoforder

 * mssql+pymssql://DESKTOP-80SJ5RC/Northwind
Done.


yearoforder,NumOrders
1996,152
1997,408
1998,270


## Orders by Year

In [23]:
%%sql
--Drill down to month and year
WITH DimOrderDate AS (
SELECT
	DISTINCT
	OrderDate,
	YEAR(OrderDate) as year,
	MONTH(OrderDate) as month,
	DAY(OrderDate) as day,
	DATENAME(MM, OrderDate) as monthname,
	DATEPART(QQ,OrderDate) as quarter
FROM 
	Orders
)

SELECT 
	month_year,
	num_orders
FROM(
SELECT 
	CONCAT(d.monthname, '-', d.year) as month_year,
	d.year,
	d.month,
	COUNT(o.OrderID) as num_orders
FROM Orders o
LEFT JOIN DimOrderDate d
ON o.OrderDate = d.OrderDate
GROUP BY CONCAT(d.monthname, '-', d.year), d.year, d.month
) as subq2
ORDER BY
	year,
	month

 * mssql+pymssql://DESKTOP-80SJ5RC/Northwind
Done.


month_year,num_orders
July-1996,22
August-1996,25
September-1996,23
October-1996,26
November-1996,25
December-1996,31
January-1997,33
February-1997,29
March-1997,30
April-1997,31


## Orders by Month and Year

In [24]:
%%sql
--Drill down to month and year
WITH DimOrderDate AS (
SELECT
	DISTINCT
	OrderDate,
	YEAR(OrderDate) as year,
	MONTH(OrderDate) as month,
	DAY(OrderDate) as day,
	DATENAME(MM, OrderDate) as monthname,
	DATEPART(QQ,OrderDate) as quarter
FROM 
	Orders
)

SELECT 
	month_year,
	num_orders
FROM(
SELECT 
	CONCAT(d.monthname, '-', d.year) as month_year,
	d.year,
	d.month,
	COUNT(o.OrderID) as num_orders
FROM Orders o
LEFT JOIN DimOrderDate d
ON o.OrderDate = d.OrderDate
GROUP BY CONCAT(d.monthname, '-', d.year), d.year, d.month
) as subq2
ORDER BY
	year,
	month

 * mssql+pymssql://DESKTOP-80SJ5RC/Northwind
Done.


month_year,num_orders
July-1996,22
August-1996,25
September-1996,23
October-1996,26
November-1996,25
December-1996,31
January-1997,33
February-1997,29
March-1997,30
April-1997,31


## Orders by Quarter

In [25]:
%%sql
--Drill down to quarter and year
WITH DimOrderDate AS (
SELECT
	DISTINCT
	OrderDate,
	YEAR(OrderDate) as year,
	MONTH(OrderDate) as month,
	DAY(OrderDate) as day,
	DATENAME(MM, OrderDate) as monthname,
	DATEPART(QQ,OrderDate) as quarter
FROM 
	Orders
)

SELECT 
	quarter_year,
	num_orders
FROM(
SELECT 
	CONCAT('Q',d.quarter, '-', d.year) as quarter_year,
	d.quarter,
	d.year,
	COUNT(o.OrderID) as num_orders
FROM Orders o
LEFT JOIN DimOrderDate d
ON o.OrderDate = d.OrderDate
GROUP BY CONCAT(d.quarter, '-', d.year), d.year, d.quarter
) as subq2
ORDER BY
	year,
	quarter

 * mssql+pymssql://DESKTOP-80SJ5RC/Northwind
Done.


quarter_year,num_orders
Q3-1996,70
Q4-1996,82
Q1-1997,92
Q2-1997,93
Q3-1997,103
Q4-1997,120
Q1-1998,182
Q2-1998,88


## Order Subtotals
something I noticed was the discount rates were not aligned based on the product unit price. This corrects for the right discount applied. I.e ProductID 11 has a unitprice of 21.00 in Products. Order Details Unit Price is 14.00 with 0% discount rate. This is incorrect

In [27]:
%%sql 
WITH corrected_orderdetails AS (
select 
	od.OrderID,
	od.ProductID,
	od.UnitPrice,
	od.Quantity,
	CASE WHEN 
		od.UnitPrice != p.UnitPrice
		THEN ROUND(1 - (od.UnitPrice/p.UnitPrice),2)
		ELSE 0 END as Discount
from [Order Details] od
LEFT JOIN Products p
ON od.ProductID = p.ProductID
)
SELECT
	OrderID,
	SUM(Subtotal) as OrderSubtotal
FROM
(
	SELECT 
		OrderID,
		ProductID,
		UnitPrice,
		Quantity,
		(UnitPrice * Quantity) AS Subtotal
	FROM corrected_orderdetails
) AS subtotals
GROUP BY
	OrderID

 * mssql+pymssql://DESKTOP-80SJ5RC/Northwind
Done.


OrderID,OrderSubtotal
10248,440.0000
10249,1863.4000
10250,1813.0000
10251,670.8000
10252,3730.0000
10253,1444.8000
10254,625.2000
10255,2490.5000
10256,517.8000
10257,1119.9000


## Orders Subtotal Plus Shipping Cost

In [30]:
%%sql 
WITH corrected_orderdetails AS (
select 
	od.OrderID,
	od.ProductID,
	od.UnitPrice,
	od.Quantity,
	CASE WHEN 
		od.UnitPrice != p.UnitPrice
		THEN ROUND(1 - (od.UnitPrice/p.UnitPrice),2)
		ELSE 0 END as Discount
from [Order Details] od
LEFT JOIN Products p
ON od.ProductID = p.ProductID
)
SELECT 
	ordersubtotals.OrderID,
	OrderSubtotal + o.Freight AS OrderSubtotal_PlusShipping
FROM (
	SELECT
		subtotals.OrderID,
		SUM(Subtotal) as OrderSubtotal
	FROM
	(
		SELECT 
			OrderID,
			ProductID,
			UnitPrice,
			Quantity,
			(UnitPrice * Quantity) AS Subtotal
		FROM corrected_orderdetails
	) AS subtotals
	GROUP BY
		subtotals.OrderID
	) AS ordersubtotals
LEFT JOIN Orders O
ON o.OrderID = ordersubtotals.OrderID

 * mssql+pymssql://DESKTOP-80SJ5RC/Northwind
Done.


OrderID,OrderSubtotal_PlusShipping
10248,472.3800
10249,1875.0100
10250,1878.8300
10251,712.1400
10252,3781.3000
10253,1502.9700
10254,648.1800
10255,2638.8300
10256,531.7700
10257,1201.8100


## Actively Sold Products by Category

In [31]:
%%sql
SELECT 
	c.CategoryName,
	COUNT(p.ProductID) AS NumberofProducts
FROM Products p
LEFT JOIN Categories c
ON c.CategoryID = p.CategoryID
Where p.Discontinued < 1
GROUP BY
	c.CategoryName

 * mssql+pymssql://DESKTOP-80SJ5RC/Northwind
Done.


CategoryName,NumberofProducts
Beverages,11
Condiments,11
Confections,13
Dairy Products,10
Grains/Cereals,6
Meat/Poultry,2
Produce,4
Seafood,12
